In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import  html
import plotly.express as px
from dash import dash_table as dt
#from dash.dependencies import Input, Output, State
from dash import Output
from dash import Input
from dash import State


from pymongo.encryption import (Algorithm,
                                ClientEncryption)
from pymongo.encryption_options import AutoEncryptionOpts

import hashlib
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson import Binary, Code
from bson.json_util import dumps, loads
import base64
import matplotlib.pyplot as plt


#CRUD Python module file name and class name
from animal_crud import AnimalShelter






###########################
# Data Manipulation / Model
###########################
#Username, password and CRUD Python module name
username = "aacuser"
#password = "monica123"

salt = os.urandom(32) # Remember this
password = "monica123"


#Method to hash password

key = hashlib.pbkdf2_hmac(
    'sha256', # The hash digest algorithm for HMAC
    password.encode('utf-8'), # Convert the password to bytes
    salt, # Provide the salt
    100000 # It is recommended to use at least 100,000 iterations of SHA-256 
)



shelter = AnimalShelter(username, password)


#Class read method  that support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))




#########################
# Dashboard Layout / View
#########################
app = dash.Dash('dash.Dash')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
   # html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('UUID: 84b80843-aafa-4dfb-935f-20832df64a85'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(
         
#Code for the interactive filtering options. Radio buttons, drop downs


    ),
    html.Div([
        html.Label(['AAC Filter Options']),
        dcc.RadioItems(
            id='radio_items_id',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain Rescue', 'value': 'MR'},
                {'label': 'Disaster Rescue', 'value': 'DR'},
                {'label': 'Reset', 'value': 'RESET'},
            ],
                
                  
            
            value='Reset',
            labelStyle={'display': 'inline-block'},           
            style={"width": "50%"}
            ),
   
       
    ]),
    
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#Features for interactive data table to make it user-friendly for your client
 
        editable=False,
        #filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
    html.Br(),
    html.Hr(),

   
# This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
             id='graph-id',
             className='col s12 m6',
            
        ),
         html.Div(
    
           id='map-id',
           className='col s12 m6',
          

         ),
                 
       
                 

        ])       
                 
])      
                             
    
              

##############################################
 # Interaction Between Components / Controller
#############################################
   


#callback method and filter for interactive data table with Mongo queries
    
@app.callback([Output('datatable-id','data'),
             Output('datatable-id','columns')],
             [Input('radio_items_id','value')])
                     
def update_dashboard(value):
    
      
    
    if value == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
    
    
        
    if value == 'WR':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
        "Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}})))
      
 
                                             
    if value == 'MR':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd",
"Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
 "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
"age_upon_outcome_in_weeks":{"$lte":156}})))
        
        
                  
    if value == 'DR':
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher",                                                                                 
     "German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male",
"age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))                                     
                                                        
                                                
        

     
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)
    

#Function loop and callback for selected items in columns    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)


def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#Function and callback for bar chart

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]   
   
)



def update_graphs(viewData):
    
    
   
    dff = pd.DataFrame.from_dict(viewData)
    return html.Div(
        [
            dcc.Graph(
                id=column,
                figure={
                    "data": [
                        {
                            "x": dff["breed"],
                            "y": dff[column] if column in dff else [],
                            "type": "bar",
                            "marker": {"color": "#0074D9"},
                        }
                    ],
                    "layout": {
                        "xaxis": {"automargin": True},
                        "yaxis": {"automargin": True},
                        "height": 250,
                        "margin": {"t": 10, "l": 10, "r": 10},
                    },
                },
            )
            for column in ["age_upon_outcome", "breed", "color"]
        ]
    )
    

    
 


 


 #Function and callback for geolocation map   
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
   
)
def update_map(viewData):
    

    dff = pd.DataFrame.from_dict(viewData)
  

    # Freeport NY is at [40.65,-73.58]
    return [    
        

                                 
       
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[40.65,-73.58], zoom=10, children=[
             dl.TileLayer(id="base-layer-id"),
             
            # Marker with tool tip and popup
            dl.Marker(position=[40.65,-73.58], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[2,9])
                ])
            ])
        ])
    ]
        
    
 


     



#Run application

if __name__ == "__main__":
    app.run_server()
    


ServerSelectionTimeoutError: SSL handshake failed: localhost:27017: [WinError 10054] An existing connection was forcibly closed by the remote host, Timeout: 30s, Topology Description: <TopologyDescription id: 62a167ed3c455d1484451c1e, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake failed: localhost:27017: [WinError 10054] An existing connection was forcibly closed by the remote host')>]>